# Particle Filter

## Introduction
In this example we show how to implement a particle filter. As an example we will use the model of a CSTR reactor. 

<img src="../images/bioreactor.png" alt="drawing" width="600"/>

The model is 

$$
\begin{align}\label{eq:temp}
\dot{x}_1(t) &= -a_1 x_1(t) + b_1r  \\
\dot{x}_2(t) &= -a_2 x_2(t) + b_2r + gu \\
\end{align}
$$

where $r = (1 - x_1(t))\exp(-E/(1 + x_2(t)))$ where $a_1,b_1,a_2,b_2,E$ and $g$ are constants. The goal is estimating the concentrations of $x_1$ and $x_2$ from noisy measurements.



## Code

In [1]:
# Add NMPC to path. NOT NECESSARY if it was installed via pip.
import sys
sys.path.append('../../')

import numpy as np

from hilo_mpc import PF
from hilo_mpc.library.models import cstr_schaffner_and_zeitz as cstr
from hilo_mpc.util.plotting import get_plot_backend, set_plot_backend

# Seed for noise
np.random.seed(0)

Initialize the model

In [2]:
# Load CSTR model
set_plot_backend('bokeh')
model = cstr()

# Set up model
model.setup(dt=.1)

Initialize the particle filter

In [3]:
# Initialize particle filter
pf = PF(model, plot_backend='bokeh', prior_editing=True, K=.02)

# Set up particle filter
pf.setup(n_samples=50)

In [4]:
# Constants
const = {
    'a_1': .2674,
    'a_2': 1.815,
    'b_1': 1.05e14,
    'b_2': 4.92e13,
    'g': 1.5476,
    'E': 34.2583
}

# Initial conditions of the model
model.set_initial_conditions([.5, 0.])
model.set_initial_parameter_values(const)


Initialize the weights and initial guesses

In [5]:
# Initial guess for the particle filter
pf.Q = [1e-4, 1e-7]
pf.R = 1e-4
pf.set_initial_guess([.4, .1], P0=[.02 * .25, .02 * .25])
pf.set_initial_parameter_values(const)

In [6]:
# Run simulations
for k in range(200):
    model.simulate(u=-.002)

    # Get noisy measurement and calculate estimates
    yk = model.solution.make_some_noise('y:f', var={'y': 1e-4})
    pf.estimate(y=yk, u=-.002)

## Plots

In [7]:

# Plots
data = {}
names = model.dynamical_state_names
for k in range(2):
    data[f'{names[k]}_pred'] = {
            'data': pf.solution['x'][k, :],
            'subplot': k
    }
data['y_pred'] = {
    'data': pf.solution['y'],
    'subplot': k + 1
}
model.solution.plot(('t', 'x_1'), ('t', 'x_2'), ('t', 'y'),
                    title=["conversion of chemical reaction", "scaled reactor temperature", "measured temperature"],
                    y_data=data,
                    output_notebook=True)
pf.solution.plot(('t', 'P_0', 'P_3'), output_notebook=True)

Loading BokehJS ...

Loading BokehJS ...

'C:\\Users\\Bruno\\AppData\\Local\\Programs\\Python\\Python37\\lib\\runpy.py'

## Final notes
HILO-MPC is developed by Johannes Pohlodek and Bruno Morabito under the supervision of Prof. Rolf Findeisen
at the  Control and cyber-physical systems laboratory, TU Darmstadt (https://www.ccps.tu-darmstadt.de/ccp) and at the
Laboratory for Systems Theory and Control, Otto von Guericke University (http://ifatwww.et.uni-magdeburg.de/syst/).